In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [15]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [17]:
def preprocess_data(df_train):
    df_train_rand = df_train.reindex(np.random.permutation(df_train.index))
    return df_train_rand

In [18]:
def preprocess_features(x_train):
    #Normalization to greyscale, reduces the effect of illumination's differences
    x_train_norm = x_train / 255.0
    #Reshaping for keras 28x28x1 3D matrices?
    x_train_norm = x_train_norm.values.reshape(-1, 28, 28, 1)
    return x_train_norm

In [19]:
def preprocess_targets(y_train):
    #Encode labels to one hot vecs
    y_train_hot = tf.keras.utils.to_categorical(y_train, num_classes=10)
    return y_train_hot

In [20]:
df_train_rand = preprocess_data(df_train)
X_train = preprocess_features(df_train_rand.drop(labels = ['label'], axis = 1).head(38000).copy())
X_val = preprocess_features(df_train_rand.drop(labels = ['label'], axis = 1).tail(6000).copy())
Y_train = preprocess_targets(df_train_rand['label'].head(38000).copy())
Y_val = preprocess_targets(df_train_rand['label'].tail(6000).copy())

In [21]:
visible = tf.keras.layers.Input(shape=(28,28,1))

In [28]:
conv1 = tf.keras.layers.Conv2D(32, kernel_size=(5,5), padding='Same', activation='relu')(visible)
conv2 = tf.keras.layers.Conv2D(32, kernel_size=(5,5), padding='Same', activation='relu')(conv1)
pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2))(conv2)
dropout1 = tf.keras.layers.Dropout(0.25)(pool1)
conv3 = tf.keras.layers.Conv2D(64, kernel_size=(3,3), padding='Same', activation='relu')(dropout1)
conv4 = tf.keras.layers.Conv2D(64, kernel_size=(3,3), padding='Same', activation='relu')(conv3)
pool2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2))(conv4)
dropout2 = tf.keras.layers.Dropout(0.25)(pool2)
flatten = tf.keras.layers.Flatten()(dropout2)
dense1 = tf.keras.layers.Dense(245, activation='relu')(flatten)
dropout3 = tf.keras.layers.Dropout(0.25)(dense1)
output_dense = tf.keras.layers.Dense(10, activation='softmax')(dropout3)

In [29]:
model = tf.keras.Model(inputs=visible, outputs=output_dense)